# FitzHugh–Nagumo neuron system

In [1]:
using ModelingToolkit, DifferentialEquations, GLMakie
using ModelingToolkit: t_nounits as t, D_nounits as D

# FitzHugh–Nagumo parameters
@parameters ε a b A ω
@variables v(t) w(t) I(t)

# Differential equations
eqs = [
    D(v) ~ v - (v^3)/3 - w + I,
    D(w) ~ ε * (v + a - b*w),
    I ~ A * sin(ω * t)
]

@mtkbuild odesys = ODESystem(eqs, t)

# Simulation settings
tspan = (0.0, 100.0)
timesteps = 0.0:0.1:100.0
u0 = [v => -1.0, w => 1.0]
p = [ε => 0.08, a => 0.7, b => 0.8, A => 1.0, ω => 0.2]

# Solve
prob = ODEProblem(odesys, u0, tspan, p)
sol = solve(prob, Tsit5(); saveat=timesteps, abstol=1e-8, reltol=1e-6)

# Extract values
ts = sol.t
vs = sol[v]
ws = sol[w]
Is = sol[I]

# Estimate dv/dt for state classification
dvs = [0; diff(vs) ./ diff(ts)]

# Classify function
function classify_state(v, dv)
    if v > 1.0
        " Spiking"
    elseif v < -0.5 && dv < 0
        " Recovering"
    elseif abs(dv) < 0.02 && abs(v) < 0.5
        " Resting"
    else
        "↔ Transition"
    end
end

# Setup figure
fig = Figure(size=(1000, 500))
ax1 = Axis(fig[1, 1], xlabel="Time", ylabel="Voltage v(t)", title="Neuron Voltage & Input Current")
ax2 = Axis(fig[1, 2], xlabel="v", ylabel="w", title="Phase Portrait (v vs w)")

# Plot voltage & current
lines!(ax1, ts, vs, color=:blue, label="v(t)")
lines!(ax1, ts, Is, color=:green, label="I(t)")
GLMakie.hlines!(ax1, [1.0], linestyle=:dot, color=:red, label="Threshold v=1")
axislegend(ax1)

# Phase portrait
lines!(ax2, vs, ws, color=:purple)

# Animation markers
frame_idx = Observable(1)
dot_orbit = scatter!(ax2, [vs[1]], [ws[1]], color=:black, markersize=8)
dot_v = scatter!(ax1, [ts[1]], [vs[1]], color=:black, markersize=8)
vline = vlines!(ax1, [ts[1]], color=:gray, linestyle=:dash)

# Labels
value_label = Label(fig[0, :], "Time: $(ts[1])   v(t): $(vs[1])   I(t): $(Is[1])", fontsize=16)
state_label = Label(fig[2, :], "State: $(classify_state(vs[1], dvs[1]))", fontsize=18, halign=:center, color=:darkblue)

# Update function
on(frame_idx) do i
    i = clamp(i, 1, length(ts))
    dot_orbit[1][] = [Point2f(vs[i], ws[i])]
    dot_v[1][] = [Point2f(ts[i], vs[i])]
    vline[1][] = [ts[i]]
    value_label.text[] = "Time: $(round(ts[i], digits=2))   v(t): $(round(vs[i], digits=3))   I(t): $(round(Is[i], digits=3))"
    state_label.text[] = "State: $(classify_state(vs[i], dvs[i]))"
end

# Record animation
record(fig, "FitzHugh-Nagumo_Neuoron_system.mp4", 1:length(ts); framerate=20) do i
    frame_idx[] = i
end


"FitzHugh-Nagumo_Neuoron_system.mp4"